# Examples

task examples

The following examples, to be expanded, highlight modes and options that
the tclean task supports.  
  
The examples below are written as scripts that may be copied and pasted
to get started with the basic parameters needed for a particular
operation. When writing scripts, it is advised that the interactive task
interface be used to view lists of sub-parameters that are relevant only
to the operations being performed. For example, setting specmode='cube'
and running inp() will list parameters that are relevant to spectral
coordinate definition, or setting niter to a number greater than zero
(niter=100) followed by inp() will list iteration control parameters.  
  
Note that all runs of tclean need the following parameters: vis,
imagename, imsize, and cell.  
  
By default, tclean will run with niter=0, making the PSF, a primary
beam, the initial dirty (or residual) image and a restored version of
the image.

&nbsp;

For examples running tclean on ALMA data, see also the CASA Guide
["Tclean and
ALMA"](https://casaguides.nrao.edu/index.php?title=TCLEAN_and_ALMA).



###  Imaging and Deconvolution Iterations

&nbsp;



#### Using Hogbom CLEAN on a single MFS image

tclean(vis='test.ms', imagename='try1', imsize=100, cell='10.0arcsec',
specmode='mfs',  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; deconvolver='hogbom',
gridder='standard', weighting='natural', niter=100 )



#### Using Multi-scale CLEAN on a Cube Mosaic image

&nbsp;  

tclean(vis='test.ms', imagename='try1', imsize=100,
cell='10.0arcsec',specmode='cube',

&nbsp; &nbsp; &nbsp; &nbsp;nchan=10, start='1.0GHz', width='10MHz',  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; deconvolver='multiscale',
scales=\[0,3,10,30\],  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; gridder='mosaic', pblimit=0.1,  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; weighting='natural', niter=100 )



####  Using W-Projection with Multi-Term MFS wideband imaging

tclean(vis='test.ms', imagename='try1', imsize=100, cell='10.0arcsec',  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; deconvolver='mtmfs',
reffreq='1.5GHz', nterms=2,  
&nbsp; &nbsp; &nbsp; &nbsp;gridder='wproject', wprojplanes=64,  
&nbsp; &nbsp; &nbsp; &nbsp;weighting='natural', niter=100 )



####  Using automasking with any type of image

tclean(vis='test.ms', imagename='try1', niter=100, ....,
usemask='auto-multithresh')

See the&nbsp;[Masks for
Deconvolution](https://casa.nrao.edu/casadocs-devel/stable/imaging/synthesis-imaging/masks-for-deconvolution)
section of CASAdocs for more information about the auto-multithresh
algorithm.

&nbsp;



### Scripting using PySynthesisImager

  
PySynthesisImager (LINK) is a python application built on top of the
synthesis tools (LINK). The operations of the tclean task can be
replicated using the following python script. Subsets of the script can
thus be chosen, and extra external methods can be inserted in between as
desired.&nbsp; After each stage, images are saved on disk. Therefore,
any modifications done to the images in between steps will be
honored.&nbsp;

&nbsp;

\#\

## (1) Import the python application layer  
  
from imagerhelpers.imager_base import PySynthesisImager  
from imagerhelpers.input_parameters import ImagerParameters  
  
\#\

## (2) Set up Input Parameters  
\#\

## - List all parameters that you need here  
\#\

## - Defaults will be assumed for unspecified parameters  
\#\

## - Nearly all parameters are identical to that in the task. Please
look at the  
\#\

## list of parameters under \_\_init\_\_ using " help ImagerParameters
" )  
  
paramList = ImagerParameters(  
msname ='DataTest/point.ms',  
field='',  
spw='',  
imagename='try2',  
imsize=100,  
cell='10.0arcsec',  
specmode='mfs',  
gridder='standard',  
weighting='briggs',  
niter=100,  
deconvolver='hogbom'  
)  
  
\#\

## (3) Construct the PySynthesisImager object, with all input
parameters  
  
imager = PySynthesisImager(params=paramList)  
  
\#\

## (4) Initialize various modules.  
\#\

## - Pick only the modules you will need later on. For example, to
only make  
\#\

## the PSF, there is no need for the deconvolver or iteration control
modules.  
  
\#\

## Initialize modules major cycle modules  
  
imager.initializeImagers()  
imager.initializeNormalizers()  
imager.setWeighting()  
  
\#\

## Init minor cycle modules  
  
imager.initializeDeconvolvers()  
imager.initializeIterationControl()  
  
\#\

## (5) Make the initial images  
  
imager.makePSF()  
imager.makePB()  
imager.runMajorCycle() \

## Make initial dirty / residual image  
  
\#\

## (6) Make the initial clean mask  
imager.hasConverged()  
imager.updateMask()  
  
\#\

## (7) Run the iteration loops  
while ( not imager.hasConverged() ):  
&nbsp;&nbsp;&nbsp; imager.runMinorCycle()  
&nbsp;&nbsp;&nbsp; imager.runMajorCycle()  
&nbsp;&nbsp;&nbsp; imager.updateMask()  
  
\#\

## (8) Finish up  
  
retrec=imager.getSummary();  
imager.restoreImages()  
imager.pbcorImages()  
  
\#\

## (9) Close tools.  
  
imager.deleteTools()

&nbsp;  
For model prediction (i.e. to only save an input model in preparation
for self-calibration, for example), use the following in step (5). The
name of the input model is either assumed to be \<imagename\>.model (or
its multi-term equivalent) or should be specified via the startmodel
parameter in step (2).  
&nbsp;

imager.predictModel()&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \

## Step (5)

  
For major cycle parallelization for continuum imaging (specmode='mfs'),
replace steps (1) and (3) with the following

&nbsp;

from imagerhelpers.imager_parallel_continuum import
PyParallelContSynthesisImager&nbsp;&nbsp;&nbsp;&nbsp;&nbsp; \

## Step
(1)  
imager =
PyParallelContSynthesisImager(params=paramList)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\

## Step (3)  
&nbsp;

For parallelization of both the major and minor cycles for Cube imaging,
replace steps (1) and (3) with the following, and include a virtual
concanenation call at the end. (However, note that for parallel Cube
imaging, if you would like to replace the minor cycle with your own code
(for example), you would have to go one layer deeper. For this, please
contact our team for assistance.)

&nbsp;

  
from imagerhelpers.imager_parallel_cube import
PyParallelCubeSynthesisImager&nbsp;&nbsp; \

## Step (1)  
imager =
PyParallelCubeSynthesisImager(params=paramList)&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\

## Step (3)  
imager.concatImages(type='virtualcopy')&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;
\

## Step (8)

&nbsp;



### Using tclean&nbsp;with ephemerides tables in CASA format

&nbsp;

When you have an ephermeris table that covers the whole observation:

tclean(vis=\['MS1.ms', 'MS2.ms', 'MS3.ms', 'MS4.ms',
'MS5.ms'\],selectdata=True,field="DES_DEEDEE",spw=\['17,19,21,23',
'17,19,21,23', '17,19,21,23', '17,19,21,23',
'17,19,21,23'\],intent="OBSERVE_TARGET\#ON_SOURCE",datacolumn="data",imagename="test_track",imsize=\[2000,
2000\],cell=\['0.037arcsec'\],phasecenter="des_deedee_ephem.tab",stokes="I")

You can check whether the ephermeris table is of the format that CASA
accepts by using the measures tool me.framecomet function:

&nbsp;

me.framecomet('des_deedee.tab')

If this tool accepts the input without complaint, then the same should
work in tclean.  
  
If the source you are tracking is one of the ten sources for which the
CASA measures tool has the ephemerides from the JPL DE200 or DE405, then
you can use their names directly:

&nbsp;

tclean(vis=\['uid\_\_\_A002_Xbc74ea_X175c.ms',
'uid\_\_\_A002_Xbc74ea_X1af4.ms', 'uid\_\_\_A002_Xbc74ea_X1e19.ms',
'uid\_\_\_A002_Xbc74ea_X20b7.ms'\],selectdata=True,field="Jupiter",spw=\['17,19,21,23',
'17,19,21,23', '17,19,21,23',
'17,19,21,23'\],intent="OBSERVE_TARGET\#ON_SOURCE",datacolumn="corrected",imagename="alltogether",imsize=\[700,
700\],cell=\['0.16arcsec'\],phasecenter="JUPITER",stokes="I")

For ALMA data mainly the correlator may have the ephemerides of a moving
source already attached to the FIELD tables of the MeasurementSets (as
it was used to phase track the source). In such special cases, you can
use the keyword "TRACKFIELD" in the phasecenter parameter, and then the
internal ephemerides will be used to track the source.

&nbsp;

tclean(vis=\['MS1.ms', 'MS2.ms', 'MS3.ms', 'MS4.ms',
'MS5.ms'\],selectdata=True,field="DES_DEEDEE",spw=\['17,19,21,23',
'17,19,21,23', '17,19,21,23', '17,19,21,23',
'17,19,21,23'\],intent="OBSERVE_TARGET\#ON_SOURCE",datacolumn="data",imagename="test_track",imsize=\[2000,
2000\],cell=\['0.037arcsec'\],phasecenter="TRACKFIELD",stokes="I")

&nbsp;